<img src="../resources/cropped-SummerWorkshop_Header.png">  

<h1 align="center">Cell Types Workshop SWDB 2017 </h1> 
<h3 align="center">Thursday, August 24, 2017</h3> 

<div style="border-left: 3px solid #000; padding: 1px; padding-left: 10px; background: #F0FAFF; ">
<h2>Drive path</h2>

<p>First things first - let's make sure your drive path is set up appropriately. You will want to un-comment only one of these options depending on which operating system you are running. You also may have to tweak the option for your specific set-up (e.g. change the drive letter for Windows).
</p>
</div>

In [ ]:
# please make sure your drive_path is set, so that the notebook can find the data files on the hard drive

# OS X
# drive_path = '/Volumes/Brain2017/data/dynamic-brain-workshop/cell_types_cache/'

# Windows (a good guess)
# drive_path = 'e:/data/dynamic-brain-workshop/cell_types_cache/'

# Linux (will vary; the following is possibly what Ubuntu will do)
# drive_path = '/media/Brain2017/data/dynamic-brain-workshop/cell_types_cache/'

# AWS
drive_path = '/data/dynamic-brain-workshop/cell_types_cache/'

<div style="border-left: 3px solid #000; padding: 1px; padding-left: 10px; background: #F0FAFF; ">
<h2>Cell Types Database</h2>

<p>
The Cell Types database contains data derived from single-cell experiments including in vitro electrophysiological recordings from cortical slices and morphological reconstructions based on single-neuron fills.</p>

<p>
The database can be accessed [on the web](http://celltypes.brain-map.org/). This is useful for browsing and familiarizing yourself with the data, but for more in-depth analyses, it is often better to access data programmatically. The [AllenSDK](https://alleninstitute.github.io/AllenSDK/) provides Python-based tools to support that approach, and this notebook will introduce you to them.<p>

</div>

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

from allensdk.core.cell_types_cache import CellTypesCache

<div style="border-left: 3px solid #000; padding: 1px; padding-left: 10px; background: #F0FAFF; ">
<h2>Create a CellTypesCache instance</h2>

<p>A CellTypesCache is used to copy and cache data from the cell types database. If you create the CellTypesCache instance with no arguments, then data is downloaded from the online repository:
</p>
</div>

In [ ]:
# ctc = CellTypesCache()

<div style="border-left: 3px solid #000; padding: 1px; padding-left: 10px; background: #F0FAFF; ">
<p>Because we have limited internet bandwidth, the data you need is already provided on your external hard drive. To ensure this local data is used instead of the online repository, create the CellTypesCache instance using the `manifest_file` argument:
</p>
</div>

In [ ]:
ctc = CellTypesCache(manifest_file=os.path.join(drive_path,'cell_types_manifest.json'))

**Documentation:**
[CellTypesCache](http://alleninstitute.github.io/AllenSDK/allensdk.core.html#allensdk.core.cell_types_cache.CellTypesCache),
[CellTypesCache.get_ephys_features()](http://alleninstitute.github.io/AllenSDK/allensdk.core.html#allensdk.core.cell_types_cache.CellTypesCache.get_ephys_features)
[CellTypesCache.get_cells()](http://alleninstitute.github.io/AllenSDK/allensdk.core.html#allensdk.core.cell_types_cache.CellTypesCache.get_cells)

                            
*Note: you can find these by using the search tool on the main [AllenSDK page](http://alleninstitute.github.io/AllenSDK/)*


<div style="border-left: 3px solid #000; padding: 1px; padding-left: 10px; background: #F0FAFF; ">
<h2>1. Using pre-computed features: Htr3a-expressing interneurons</h2>

<p>
Let's say we're interested in looking at interneuron diversity in the mouse visual cortex. We know that interneurons can be divided into three major types based on the expression of specific molecular markers - some interneurons express parvalbumin (Pvalb), others express somatostatin (Sst), and most of the rest express the ionotropic serotonin receptor 5HT3a (Htr3a).
</p>
</div>

<img src="../resources/Neocortical Interneurons.png">  

<div style="border-left: 3px solid #000; padding: 1px; padding-left: 10px; background: #F0FAFF; ">
<p>
We'll start by looking at the Htr3a-expressing interneurons, which have two major subpopulations - those that express vasointestinal peptide (Vip), and those that don't. Many of the non-Vip cells express neuron derived neurotrophic factor (Ndnf). In the Cell Types Database, we have cells that are labeled by transgenic lines driven by Htr3a-Cre, as well as cells with reporter expression driven by Vip-Cre and Ndnf-Cre. We can look at the properties of those cells to see if there are characteristic differences among these populations. 
</p>
<p>
Let's look at pre-computed features of these cells. To access them, we'll use SDK components that interact with the database via its API.</p>
</div>

<div style="background: #DFF0D8; border-radius: 3px; padding: 10px;">
<p><strong>Exercise 1.1:</strong></p>

<p>Use the function `ctc.get_cells()` and specify the reporter_status to get all the cre reporter positive cells (we only want the cells actually labeled by the transgenic lines we're investigating). Store them in a variable named `cells`. How many cells are in the database? Does this match the number reported on the website? </p>

</div>

<div style="border-left: 3px solid #000; padding: 1px; padding-left: 10px; background: #F0FAFF; ">

<h2> Selecting specific cell populations </h2>

Now we'd like to just select the cells for particular transgenic lines (i.e., Htr3a, Vip, and Ndnf). The full names of those driver lines are:
<ul>
<li> Htr3a-Cre_NO152 </li>
<li> Vip-IRES-Cre </li>
<li> Ndnf-IRES2-dgCre </li>
</ul>

<p>
If we look at an entry in our `cells` list, we can see that there are several fields that are of use to us for this task: e.g. the 'transgenic_line' key allows us to select for Cre line, 'id' is the specimen_id for this cell that allows us to connect between datasets and with metadata, and 'dendrite_type' gives us information about the presence of spines which strongly covaries with excitatory/inhibitory neuron type.
</p></div>

In [ ]:
example_cell = cells[100] # cells is a list of dictionaries
print(example_cell.keys())

In [ ]:
# The specimen ID - this is a handle to refer to a specific cell in the database
print("Specimen ID:", example_cell["id"])

# The transgenic line - the driver of the mouse from which the cell was recorded
print("Transgenic line:", example_cell["transgenic_line"])

# The dendrite type - this is whether or not a given cell has dendritic spines (generally a 
# feature of excitatory neurons)
print("Dendrite type:", example_cell["dendrite_type"])

<div style="background: #DFF0D8; border-radius: 3px; padding: 10px;">
<p><strong>Exercise 1.2:</strong></p>

<p>From the `cells` list, select only the cells that are from the Htr3a-Cre_NO152 line and are not spiny. Store their specimen IDs in a new list called `htr3a_ids`. How many of these cells are there?</p>

</div>

<div style="border-left: 3px solid #000; padding: 1px; padding-left: 10px; background: #F0FAFF; ">
<h3> Pandas DataFrame is useful for selecting and filtering data </h3>
</div>

**Documentation:**
[pandas.DataFrame()](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.html),
[Intro to pandas data structures](https://pandas.pydata.org/pandas-docs/stable/dsintro.html)


In [ ]:
import pandas as pd

<div style="background: #DFF0D8; border-radius: 3px; padding: 10px;">
<p><strong>Exercise 1.3:</strong></p>

<p>Make a DataFrame (`df_cells`) from the `cells` list of dictionaries. How many rows and columns does this dataframe contain?</p>

</div>

<div style="background: #DFF0D8; border-radius: 3px; padding: 10px;">
<p>**Exercise 1.4:**</p>

<p>Make a new dataframe `df_vip` by selecting from the dataframe `df_cells` only the cells that are Vip-IRES-Cre+ and are not spiny. Print id values for these cells. </p>

</div>

<div style="background: #DFF0D8; border-radius: 3px; padding: 10px;">
<p>**Exercise 1.5:**</p>

<p>Make a new dataframe `df_tg_cells` from `df_cells` which includes all cells from the three transgenic lines of interest here: Htr3a-Cre_NO152, Vip-IRES-Cre, Ndnf-IRES2-dgCre. Hint: use the `isin` method. Additionally, only select cells that are not 'spiny'.</p>

</div>

<div style="border-left: 3px solid #000; padding: 1px; padding-left: 10px; background: #F0FAFF; ">
<p>
Now we have a table (`df_tg_cells`) with all cells labeled by our three Cre lines. Next, we'll get ephys features for the cells in the Cell Types database and extract the properties for the cells in `df_tg_cells`.
</p>
</div>

<div style="border-left: 3px solid #000; padding: 1px; padding-left: 10px; background: #F0FAFF; ">

<h3> Get Ephys Features from CellTypesCache object</h3>
<p>
We can get ephys features as a list of dictionaries. The returned data provides a record of feature values for each cell in the database, and additionally includes essential metadata about the cell (specimen_id) and experiment (id).
</p>
</div>

In [ ]:
ephys_features_dict = ctc.get_ephys_features()
ephys_features_dict[0]

<div style="border-left: 3px solid #000; padding: 1px; padding-left: 10px; background: #F0FAFF; ">
<p>
Now we have a table (`df_tg_cells`) with all cells labeled by our three Cre lines. Next, we'll get ephys features for the cells in the Cell Types database and extract the properties for the cells in `df_tg_cells`.
</p>
</div>

In [ ]:
ephys_features_df = pd.DataFrame.from_records(ephys_features_dict)
ephys_features_df.head(1)

<div style="border-left: 3px solid #000; padding: 1px; padding-left: 10px; background: #F0FAFF; ">

<h2> Merge `df_tg_cells` with `ephys_features_df` </h2>

<p>Critical information: 
<ul> 
<li>The merge operation will generate an expanded table (new columns) keyed to the specimen_id </li>
<li> The 'id' column of `df_tg_cells` corresponds to 'specimen_id' column of `ephys_features_df` </li>
<li> how = 'left' will include only cells in left dataframe </li>
<li> Following merge, each row (i.e. cell) now includes ephys features as additional columns</li> </p>
</div>

**Documentation:**
[pandas.DataFrame.merge()](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.merge.html)

In [ ]:
df = pd.merge(df_tg_cells,ephys_features_df,left_on='id',right_on='specimen_id',how='left')
len(df_tg_cells), len(df)

<div style="border-left: 3px solid #000; padding: 1px; padding-left: 10px; background: #F0FAFF; ">
<p>
The dataframe `df` now includes metadata and ephys features for each cell
</p>
</div>

In [ ]:
df.head(2)

<div style="border-left: 3px solid #000; padding: 1px; padding-left: 10px; background: #F0FAFF; ">
<p>Add column to `df` to store shorthand label for trangenic_line
</p>
</div>

In [ ]:
df['tg_label'] = [x.split('-')[0] for x in df.transgenic_line.values]

In [ ]:
df.tg_label.head()

<div style="border-left: 3px solid #000; padding: 1px; padding-left: 10px; background: #F0FAFF; ">

<h3> Examining an electrophysiological feature </h3>

<p>
Let's start by looking at something simple, like the input resistance (found in column `ri`). This reflects how much a cell will change its membrane potential for a given amount of current of current coming into the cell - cells with higher input resistances will experience a greater change in the membrane potential.

Since we have the features in a `DataFrame`, we can use the pandas plotting function to visualize the data in a histogram. </p>
</div>

In [ ]:
df[df.tg_label=='Htr3a']['ri'].plot(kind="hist", bins=np.linspace(0, 500, 25))

**Documentation:**
[pandas.Series.plot()](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.Series.plot.html)


<div style="border-left: 3px solid #000; padding: 1px; padding-left: 10px; background: #F0FAFF; ">

<h3> Viewing individual cells </h3>

<p>
The distribution looks multimodal - there is a peak around 100 MOhms and another around 250 MOhms. </p>

<p> Let's split up those two peaks and look at views of some individual cells. On the website, we can summarize some of the electrophysiological and morphological properties of the cells with small thumbnail representations. These thumbnails are accessible via the website, as long as you have the cell's specimen ID. </p>

<p>Not all cells have a morphological reconstruction; those that don't can't have a thumbnail. So we will need to only look at cells that have a reconstruction for this task. </p>

</div>

In [ ]:
# These are the URLs for getting the thumbnails - we need to provide a specimen ID
url_morph = "http://celltypes.brain-map.org/mouse/thumbnail/morphology?id="
url_ephys = "http://celltypes.brain-map.org/mouse/thumbnail/ephys_trace?id="

# We need these objects to view PNG files from the web inside the notebook
from IPython.display import Image, display

In [ ]:
# Let's get the specimen_id of the first cell in the dataframe.
unit_id = 50
specimen_id = df.specimen_id[unit_id]

# ...and plot its ephys thumbnail
url = url_ephys + str(specimen_id)
display(Image(url=url))

# We also can check whether it has a reconstruction...
print("Has reconstruction?", df.has_reconstruction[unit_id])

# ..and if so, show its morphology thumbnail

if df.has_reconstruction[unit_id]:
    url = url_morph + str(specimen_id)
    display(Image(url=url))

<div style="border-left: 3px solid #000; padding: 1px; padding-left: 10px; background: #F0FAFF; ">

The reconstruction thumbnails show the reconstructed axons in blue and the dendrites in red. Not all neurons have their axons reconstructed, so a lack of blue doesn't usually mean the axon is actually small - it just means the data were not available. The morphology is shown to fill the thumbnail, and the histogram on the right shows where those processes fall in the full cortical depth (top is near the pia).
</div>

<div style="background: #DFF0D8; border-radius: 3px; padding: 10px;">
<p>**Exercise 1.6:**</p>

<p>Get the IDs of the Htr3a cells that both (1) have an input resistance below 150 MOhm and (2) have a morphological reconstruction. How many cells meet these criteria? Show the ephys and morphology thumbnails of those cells.
</p>

<p>When you get the results, look at the cells - which traces look similar to each other? Which are different from others? Is there any correspondence between the morphology and physiology? Can you imagine other features that may separate these cells?
</div>

<div style="border-left: 3px solid #000; padding: 1px; padding-left: 10px; background: #F0FAFF; ">

<h3> Comparing cell populations </h3>

<p>Let's see if the two peaks in the Htr3a input resistance distribution are consistent with the Vip+ and Ndnf+ subpopulations of Htr3a+ interneurons.</p>

</div>

<div style="background: #DFF0D8; border-radius: 3px; padding: 10px;">
<p>**Exercise 1.7:**</p>

<p>Compare the input resistance histograms of the Htr3a, Vip, and Ndnf cells. What relationship do you see? Can you think of ways of further investigating or refining this (very preliminary) analysis?
</p>
</div>

<div style="border-left: 3px solid #000; padding: 1px; padding-left: 10px; background: #F0FAFF; ">

<h3> Exploring additional precomputed features </h3>

<p>The Cell Types database includes ~50 precomuted electrophysiological features for each cell. Some of these features are correlated with one another and this can be seen by making pairwise scatter plots.</p>
</div>

In [ ]:
ephys_features_df.columns

<div style="border-left: 3px solid #000; padding: 1px; padding-left: 10px; background: #F0FAFF; ">

Plot two features againts one another
</div>

In [ ]:
plt.plot(df['threshold_i_ramp'],df['ri'],'o')
plt.xlabel('threshold_i_ramp')
plt.ylabel('ri')

<div style="border-left: 3px solid #000; padding: 1px; padding-left: 10px; background: #F0FAFF; ">

Use <strong>seaborn</strong> package to easily make a grid of pairwise feature plots

</div>

In [ ]:
import seaborn as sns

In [ ]:
# List of features to include in pairwise grid
features = ['adaptation',
            'avg_isi',
            'ri',
            'vrest',
            'threshold_i_ramp',
            'slow_trough_t_long_square',
            'tg_label']

df_tmp = df[features]
df_tmp = df_tmp.dropna() # Drop rows that include NaN value in any column
sns.pairplot(df_tmp,hue='tg_label')

<div style="background: #DFF0D8; border-radius: 3px; padding: 10px;">
<p><strong>Exercise 1.8:</strong></p>

<p>Make a new dataframe that contains cell data for the entire database (hint: merge `df_cells` with `ephys_features_df`). Make a scatter plot of features with 'spiny' and 'aspiny' cells shown in different colors. There are ~50 precomputed features to choose from--as starting point plot pairwise combinations of these features:
<ul>
<li>upstroke_downstroke_ratio_ramp</li>
<li>ri</li>
<li>threshold_i_ramp</li>
<li>slow_trough_t_long_square</li>
<li>log_avg_isi (Derive this new feature by adding a column to the dataframe and storing the computation log(avg_isi))</li></ul>
</p>
</div>

<div style="border-left: 3px solid #000; padding: 1px; padding-left: 10px; background: #F0FAFF; ">
<h2>2. Analyzing electrophysiology sweeps</h2>

<p>
Now we'll look at a different class of interneurons - the somatostatin-positive (Sst) cells. As we've seen, there are many features pre-calculated in the Allen Cell Types Database, but it may not have everything we're interested in looking at from the original data. For example, let's look at two Sst cells where we think we see a difference in the shape of the membrane potential response in between action potentials.
</p>
</div>

In [ ]:
example_sst_ids = [475049291, 313862274]
for specimen_id in example_sst_ids:
    print(specimen_id)
    url = url_ephys + str(specimen_id)
    display(Image(url=url))

<div style="border-left: 3px solid #000; padding: 1px; padding-left: 10px; background: #F0FAFF; ">

<p>
We may not have a feature that illustrates this difference in the "sharpness" of the interspike interval shape. If we want to calculate other electrophysiology features, we'll need to look at where the data are stored - the NWB (Neurodata Without Borders) files. Each recorded cell has its own NWB file.
</p>
</div>

In [ ]:
cells = ctc.get_cells(reporter_status=["cre reporter positive"])

In [ ]:
## this saves the NWB file to 'specimen_464212183/ephys.nwb' (or just gives a handle if you already saved it)
specimen_id = 464212183
data_set = ctc.get_ephys_data(specimen_id)

**Documentation:**
[CellTypesCache](http://alleninstitute.github.io/AllenSDK/allensdk.core.html#allensdk.core.cell_types_cache.CellTypesCache),
[CellTypesCache.get_ephys_data()](http://alleninstitute.github.io/AllenSDK/allensdk.core.html#allensdk.core.cell_types_cache.CellTypesCache.get_ephys_data)

<div style="border-left: 3px solid #000; padding: 1px; padding-left: 10px; background: #F0FAFF; ">

<p>
The electrophysiological data in a Cell Types NWB file is organized by "sweeps." These sweeps represent specific periods during which a single stimulus is applied to the cell and the response is recorded. Different types of stimuli can be applied to the cell, and similar stimuli are grouped by type on the web app. For example, one-second long step current injections are grouped together under the name "Long Squares." We refer to sweeps by their number, which are integers indicating the order in which the sweeps were collected.</p>

<p>
There are typically periods of time during an experiment between sweeps; this may be due to the experimentalist making some on-line adjustments, or may be intentional to allow the cell to return to some baseline after an intense stimulus. </p>

<p>
Returning to the example, we now have an object called `data_set` that provides access to the data in the NWB file through built-in methods. We can access sweep-level data like this:</p>
</div>

In [ ]:
sweep_number = 30
sweep_data = data_set.get_sweep(sweep_number)

**Documentation:**
[NwbDataSet](http://alleninstitute.github.io/AllenSDK/allensdk.core.html#allensdk.core.nwb_data_set.NwbDataSet), [NwbDataSet.get_sweep()](http://alleninstitute.github.io/AllenSDK/allensdk.core.html#allensdk.core.nwb_data_set.NwbDataSet.get_sweep)

<div style="background: #DFF0D8; border-radius: 3px; padding: 10px;">
<p>**Exercise 2.1:**</p>

<p>`data_set.get_sweep()` returns a dict with several key/value pairs. What are the data types of these values?</p>

</div>

<div style="border-left: 3px solid #000; padding: 1px; padding-left: 10px; background: #F0FAFF; ">

<h3>Getting sweeps from an NWB file</h3>

<p>Most of the sweeps in the Cell Types Database are recorded in current-clamp mode, meaning that a current stimulus is injected into the recorded cell, and a voltage response is recorded. The current data is stored in the NWB file in amperes, and the voltage data is in volts. However, it is usually more convenient to work with the data in picoamps and millivolts.</p>
</div>

<div style="background: #DFF0D8; border-radius: 3px; padding: 10px;">
<p>**Exercise 2.2:**</p>

<p>Write code to put the stimulus (current) and response (voltage) from the sweep into variables `i` and `v`, respectively. Convert their units to pA and mV (the raw data are expressed in unscaled units).</p>
</div>

<div style="border-left: 3px solid #000; padding: 1px; padding-left: 10px; background: #F0FAFF; ">

The time points are not included in the NWB file as a waveform, but we can generate one based on the sampling rate:
</div>

In [ ]:
sampling_rate = sweep_data["sampling_rate"] # in Hz

<div style="background: #DFF0D8; border-radius: 3px; padding: 10px;">
<p>**Exercise 2.3:**</p>

<p>Create a new array `t` that is the same length as `v` and `i` and contains the correct time points (in seconds) according to the sampling rate.</p>

<p>See: [numpy.arange()](http://docs.scipy.org/doc/numpy/reference/generated/numpy.arange.html)</p>
</div>

<div style="border-left: 3px solid #000; padding: 1px; padding-left: 10px; background: #F0FAFF; ">

Finally, we can plot the data to see what it looks like.
</div>

In [ ]:
fig, axes = plt.subplots(2, 1, sharex=True)
axes[0].plot(t, v, color='black')
axes[1].plot(t, i, color='gray')
axes[0].set(ylabel="mV")
axes[1].set(ylabel="pA", xlabel="seconds")

<div style="border-left: 3px solid #000; padding: 1px; padding-left: 10px; background: #F0FAFF; ">

<h3>Calculating features of the ephys data</h3>

<p>
Plotting the data is great, but if we want to do some analysis on things like spike times, spike shapes, etc., we want to go through and extract these features from our raw data. This can be accomplished by using the feature extraction tools in the SDK.</p>
</div>

In [ ]:
from allensdk.ephys.ephys_extractor import EphysSweepFeatureExtractor

# This finds and analyzes features of spikes
sweep_ext = EphysSweepFeatureExtractor(t=t, v=v, i=i, start=1.02, end=2.02)
sweep_ext.process_spikes()

# We can pull out some information from the sweep_ext object once that processing is done
print("Avg spike threshold: {:.1f} mV".format(sweep_ext.spike_feature("threshold_v").mean()))
print("Avg spike width: {:.2f} ms".format(1e3 * np.nanmean(sweep_ext.spike_feature("width"))))

**Documentation:**
[EphysSweepFeatureExtractor](http://alleninstitute.github.io/AllenSDK/allensdk.ephys.html#allensdk.ephys.ephys_extractor.EphysSweepFeatureExtractor)

<div style="border-left: 3px solid #000; padding: 1px; padding-left: 10px; background: #F0FAFF; ">

The method `spike_feature()` returns a NumPy array of features for each spike. You pass it the name of the feature that you want. Features that can't be calculated for a given spike are set to `NaN`.

We can take a look at all the properties calculated for each spike by the extractor:</div>

In [ ]:
sweep_ext.spike_feature_keys()

<div style="border-left: 3px solid #000; padding: 1px; padding-left: 10px; background: #F0FAFF; ">

We can look at when the spikes occur by looking at the `threshold_t` property (i.e., time of spike threshold).</div>

In [ ]:
spike_times = sweep_ext.spike_feature("threshold_t")

print(spike_times[:5])  # print just the first 5 spike times

<div style="border-left: 3px solid #000; padding: 1px; padding-left: 10px; background: #F0FAFF; ">

Spike times and numbers of spikes are actually something that have already been calculated and stored elsewhere - if you're looking to get numbers of spikes for a big set of cells and/or sweeps, it's probably faster to get them via the database rather than re-calculating them all.</div>

In [ ]:
# Get a dictionary of data about each sweep
sweep_info = ctc.get_ephys_sweeps(specimen_id)
sweep_info[0]

In [ ]:
# Find the sweeps that use the "Long Square" protocol
ls_sweeps = [s for s in sweep_info
             if s["stimulus_name"] == "Long Square"]

In [ ]:
# Print the number of spikes and stimulus amplitude
for sweep in ls_sweeps:
    print(np.round(sweep["stimulus_absolute_amplitude"]), sweep["num_spikes"])

**Documentation:**
[CellTypesCache.get_ephys_sweeps()](http://alleninstitute.github.io/AllenSDK/allensdk.core.html#allensdk.core.cell_types_cache.CellTypesCache.get_ephys_sweeps)

<div style="border-left: 3px solid #000; padding: 1px; padding-left: 10px; background: #F0FAFF; ">

<h3>Getting the interspike interval (ISI) membrane potential shape for Sst cells</h3>

<p>Now we have the tools we need to derive ISI membrane potential information from our Sst (Cre line name: Sst-IRES-Cre) cells.</p>

<p>Let's start by getting the cell IDs for Sst cells from the complete list of `cells`.</p>

</div>

<div style="background: #DFF0D8; border-radius: 3px; padding: 10px;">
<p>**Exercise 2.4:**</p>

<p>Store the IDs of the non-spiny, Sst+ cells in a list called `sst_ids`. How many Sst cells are there?</p>

</div>

<div style="border-left: 3px solid #000; padding: 1px; padding-left: 10px; background: #F0FAFF; ">


<p>The shape of the ISI can change depending on how fast the cell is firing, so if we want to be consistent across cells, we should look for sweeps of a similar firing rate. But what level should that be?</p>

<p>We can look to the data to give us some idea.</p>

</div>

<div style="border-left: 3px solid #000; padding: 1px; padding-left: 10px; background: #F0FAFF; ">


<p>For the first cell in the `sst_ids` list, plot the number of spikes against the stimulus amplitude for the sweeps that:</p>

<ul>
<li>Are "Long Square" type
<li>Have a positive stimulus amplitude
<li>Have at least one spike
</ul>
</div>

In [ ]:
example_id = sst_ids[20]
dataset = ctc.get_ephys_data(example_id)

sweep_info = ctc.get_ephys_sweeps(example_id)

ls_sweeps = [s for s in sweep_info
             if s["stimulus_name"] == "Long Square"
             and s["num_spikes"] > 0
             and s["stimulus_absolute_amplitude"] > 0]

amps = [np.round(s["stimulus_absolute_amplitude"]) for s in ls_sweeps]
amps -= np.min(amps) # Align data to minimum stimulus that evokes a response
n_spikes = [s["num_spikes"] for s in ls_sweeps]

plt.plot(amps, n_spikes, 'o')

<div style="background: #DFF0D8; border-radius: 3px; padding: 10px;">
<p>**Exercise 2.5:**</p>

<p>Now, do the same thing for every Sst cell, and plot them all on the same plot. Use transparency (`alpha`) to help with overlaying so much data.
</div>

<div style="background: #DFF0D8; border-radius: 3px; padding: 10px;">
<p>**Exercise 2.6:**</p>

<p>Create a `dict` called `sweep_matches` and populate it with the sweep_number of the long square sweep that has a firing rate closest to 10 spikes/sec (use the specimen ID as the key).</p>
<p>However, don't include any sweeps that have less than 3 spikes (so for each sweep have at least 2 ISI events to average).</p></div>

**Documentation:**
[numpy.abs()](https://docs.scipy.org/doc/numpy/reference/generated/numpy.absolute.html), [numpy.argmin()](https://docs.scipy.org/doc/numpy/reference/generated/numpy.argmin.html)

<div style="border-left: 3px solid #000; padding: 1px; padding-left: 10px; background: #F0FAFF; ">


<p>We can now use the feature extractor to find the features of the spikes that will let us identify the interspike intervals. We'll define it as extending from the end of a spike (found in the feature called "`fast_trough_index`")  and the beginning of the next spike (found in the feature called "`threshold_index`".</p>

<p>
Here's a function that will calculate the average ISI. Look it over carefully and try to figure out what is happening at each step.</p>
</div>

In [ ]:
from scipy import stats

def average_isi_trajectory(dataset, sweep_num, n_points=100):
    """Average the membrane potential trajectory between spikes
    
    Parameters
    ----------
    dataset: NwbDataSet
        Data set with the ephys sweeps
    sweep: int
        Sweep number
    n_points: int
        Number of points for the resampled trace (default 100)
        
    Returns
    -------
    avg_isi: ndarray
        Averaged interspike interval membrane potential
    """
    sweep = dataset.get_sweep(sweep_num)
    
    # Extract the data from the sweep
    v = sweep["response"] * 1e3 # to mV
    t = np.arange(len(v)) / sweep["sampling_rate"] # in seconds
    
    # For Long Squares, the stimulus begins at 1.02 s and ends at 2.02 s
    ext = EphysSweepFeatureExtractor(v=v, t=t, start=1.02, end=2.02)
    
    # TODO: there's a bug in the SDK extractor we need to track down
    try:
        ext.process_spikes()
    except:
        return None
    
    # Get the starts and ends of the spikes
    threshold_indexes = ext.spike_feature("threshold_index").astype(int)
    fast_trough_indexes = ext.spike_feature("fast_trough_index").astype(int)

    # We'll store the arrays in a list so we can average them afterwards
    isis = [] 
    
    # Get the end of a spike and the start of the next one spike
    # Note the offsets in indexing of the fast_trough_indexes and threshold_indexes
    # We're using zip() to pair up the appropriate values
    for start, end in zip(fast_trough_indexes[:-1], threshold_indexes[1:]):
        v_slice = v[start:end]
        t_slice = t[start:end]
        
        # We use scipy.stats.binned_statistic() to resample to n_points
        binned, _, _ = stats.binned_statistic(t_slice, v_slice, bins=n_points)
        isis.append(binned)
    
    # We put the different ISIs on top of each other with np.vstack()
    # then average down the column with mean(axis=0)
    avg_isi = np.vstack(isis).mean(axis=0)
    return avg_isi


**Documentation:**
[zip()](https://docs.python.org/2/library/functions.html#zip), [scipy.stats.binned_statistic()](https://docs.scipy.org/doc/scipy-0.19.0/reference/generated/scipy.stats.binned_statistic.html),
[numpy.vstack()](https://docs.scipy.org/doc/numpy/reference/generated/numpy.vstack.html)

<div style="background: #DFF0D8; border-radius: 3px; padding: 10px;">
<p>**Exercise 2.7:**</p>

<p>Use the above function to populate a `dict` called `isi_traces` (keyed on specimen IDs) with the average ISI from the sweeps in `sweep_matches`. Plot them on top of each other (use transparency to help with overplotting).</p></div>

<div style="background: #DFF0D8; border-radius: 3px; padding: 10px;">
<p>**Exercise 2.8:**</p>

<p>You may be able to see some differences in the ISI shape, but, again, aligning the data may improve your ability to see patterns. Plot these traces again, but normalize the voltage so that they all start at 0 and end at 1.</p></div>

<div style="background: #DFF0D8; border-radius: 3px; padding: 10px;">
<p>**Exercise 2.9:**</p>

<p>This is the last exercise, and it is intentionally open-ended.</p>

<p>Try to think of a feature (i.e. a number or maybe a set of numbers) that would capture the differences you see in the ISI shape. You may need to do something like average a particular interval or take the derivative of the trace.</p>

<p>Implement your feature calculation in a function and apply it to the traces. See if it separates the cells as you think it should - you can plot the ephys thumbnails as we did above as one way of checking for that.</p>

<p>Future directions you can go could be things like:</p>

<ul>
<li>Does this ISI shape difference correlate with other things I know about the cell (its depth in cortex? its morphology? other electrophysiological features?)?
<li>How much does the feature you created change within a given cell as the stimulus amplitude changes?
<li>Are differences more pronounced at lower spike rates? Higher spike rates?
</ul>

</div>

<div style="border-left: 3px solid #000; padding: 1px; padding-left: 10px; background: #F0FAFF; ">

<h2>Project Ideas</h2>

<p>Here are some ideas to get you started in thinking about your projects.
<li> Examine how ephys / morph features vary across Cre-line and/or cortical layer
<li> Attempt to classify cells using ephys data/features (e.g., use machine learning to predict Cre line)
<li> Perform classification of cells based on ephys features extracted from the experimental data and then from biophysical models; compare the two taxonomies.
<li> Engineer new features or feature combinations from the raw data and see how they affect classification results. (For example, additional quantitative measures of evoked firing with sustained depolarization).
<li> Investigate correlations between ephys features and morphological features (for the latter, create your own as necessary).
<li> Look at whether truncation of apical dendrites is linked to differences in any ephys features
<li> Investigate whether electrophysiological cell features systematically vary across the anterior-posterior or medial-lateral dimensions of the primary visual cortex. 
<li> Characterize the temporal precision and reliability of spiking responses to repeated sweeps of the noise stimulus. How consistent are evoked spike trains within vs between cells? Determine whether different cell populations (e.g. different Cre lines) vary consistently in their reliability/precision.  Do any electrophysiological or morphological features correlate with the temporal reliability of spiking.
<li> Our electrophysiology recordings are 30 minutes long. We apply stringent versions of the most common cited quality control parameters in the neuroscience literature (based on access resistance, bias current, etc). But these are largely used for historical reasons, and there still could be measurable changes in the cell’s extracted features within the 30 minutes of recording (drift or perhaps variability). Determine whether features extracted from the beginning vs later in the recording are, and whether certain Cre lines are more sensitive to this problem than others.
<li> Calculate “dendrite density clouds” after orienting and averaging morphologies
<li> Look for correlations between ephys data and Gene Expression and Taxonomy in V1
<li> Connect Brain Observatory functional properties to ephys intrinsic properties

</div>

<div style="border-left: 3px solid #000; padding: 1px; padding-left: 10px; background: #F0FAFF; ">

<h2>Functions Ideas</h2>

<li> cells for a given cre line
<li> plot a set of cell thumbnails in a notebook via the web
<li> v, i, and t for a given sweep & data set

</div>